# PyTorch Lightning baseline template

by Andrés Muñoz-Jaramillo

This notebook is meant to act as a template to train and use a simple regression model to define a baseline that can be compared with a DS application.

It focuses on the concept of defining a PyTorch model, a PyTorch lightning training loop and the deffinition of metrics of performance.

This notebook assumes familiarity with the concepts of datasets and dataloaders contained in the **_0_dataset_dataloader_template.ipynb_**

## Set your cuda visible device

**IMPORTANT:** Since we are sharing resources, please make sure that the cuda visible device you put here is the one assigned to your team and your machine.   

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "2"

Here we initalize variables related to Weights and Biases, our online logging system to ensure they are user specific

In [2]:
# Make sure wandb logs are stored in a user-specific directory
# Set writable directories
os.environ["WANDB_DIR"] = "./wandb/wandb_logs"
os.environ["WANDB_CACHE_DIR"] = "./wandb/wandb_cache"
os.environ["WANDB_CONFIG_DIR"] = "./wandb/wandb_config"
# Optional:
os.environ["TMPDIR"] = "./wandb/wandb_tmp"

# Ensure directories exist (optional, wandb usually creates them)
os.makedirs(os.environ["WANDB_DIR"], exist_ok=True)
os.makedirs(os.environ["WANDB_CACHE_DIR"], exist_ok=True)
os.makedirs(os.environ["WANDB_CONFIG_DIR"], exist_ok=True)
os.makedirs(os.environ["TMPDIR"], exist_ok=True)

In [3]:

import sys
from torch.utils.data import DataLoader

import torch
import yaml

import lightning as L
from lightning.pytorch.callbacks import ModelCheckpoint
from lightning.pytorch.loggers import CSVLogger, WandbLogger

# Append base path.  May need to be modified if the folder structure changes.
# It gives the notebook access to the wokshop_infrastructure folder.
sys.path.append("../../")
 
# Append Surya path. May need to be modified if the folder structure changes.
# It gives the notebook access to surya's release code.
sys.path.append("../../Surya")

from surya.utils.data import build_scalers  # Data scaling utilities for Surya stacks

torch.set_float32_matmul_precision('medium')



## Download scalers
Surya input data needs to be scaled properly for the model to work and this cell downloads the scaling information.

- If the cell below fails, try running the provided shell script directly in the terminal.
- Sometimes the download may fail due to network or server issues—if that happens, simply re-run the script a few times until it completes successfully.

In [4]:
!sh download_scalers.sh

==> Checking assets directory at: /home/shahbahauddin/surya_workshop/downstream_apps/template/assets
==> Downloading scalers.yaml from nasa-ibm-ai4science/core-sdo
Fetching 1 files: 100%|██████████████████████████| 1/1 [00:00<00:00, 837.19it/s]
/home/shahbahauddin/surya_workshop/downstream_apps/template/assets
✓ Done. File is located at: /home/shahbahauddin/surya_workshop/downstream_apps/template/assets/scalers.yaml


## Load configuration

Surya was designed to read a configuration file that defines many aspects of the model
including the data it uses we use this config file to set default values that do not
need to be modified, but also to define values specific to our downstream application

In [5]:
# Configuration paths - modify these if your files are in different locations
config_path = "./configs/config.yaml"

# Load configuration
print("📋 Loading configuration...")
try:
    config = yaml.safe_load(open(config_path, "r"))
    config["data"]["scalers"] = yaml.safe_load(open(config["data"]["scalers_path"], "r"))
    print("✅ Configuration loaded successfully!")
except FileNotFoundError as e:
    print(f"❌ Error: {e}")
    print("Make sure config.yaml exists in your current directory")
    raise

scalers = build_scalers(info=config["data"]["scalers"])

📋 Loading configuration...
✅ Configuration loaded successfully!


## Define Downstream (DS) datasets

This child class takes as input all expected HelioFM parameters, plus additonal parameters relevant to the downstream application.  Here we focus in particular to the DS index and parameters necessary to combine it with the HelioFM index.

Another important component of creating a dataset class for your DS is normalization.  Here we use a log normalization on xray flux that will act as the output target.  Making log10(xray_flux) strictly positive and having 66% of its values between 0 and 1

In this case we will define both a training and a validation dataset using the indices pointed at in the config

**_Important:  In this notebook we sets max_number_of_samples=6 to potentially avoid going through the whole dataset as we explore it.  Keep in mind this for the future in case the database seems smaller than you expect_**


In [6]:
from downstream_apps.template.datasets.template_dataset import FlareDSDataset
from downstream_apps.template.datasets.eve_dataset import EVESpectraDSDataset

In [12]:
# Path to your downstream spectra CSV
ds_spectra_csv_path = "./euv_data/sample_6k_eve_spectra.csv"

# Train dataset (EVE)
train_dataset = EVESpectraDSDataset(
    #### All these lines are required by the parent HelioNetCDFDataset class
    # (You stated these must match the template exactly)
    index_path=config["data"]["train_data_path"],
    time_delta_input_minutes=config["data"]["time_delta_input_minutes"],
    time_delta_target_minutes=config["data"]["time_delta_target_minutes"],
    n_input_timestamps=config["model"]["time_embedding"]["time_dim"],
    rollout_steps=config["rollout_steps"],
    channels=config["data"]["channels"],
    drop_hmi_probability=config["drop_hmi_probability"],
    use_latitude_in_learned_flow=config["use_latitude_in_learned_flow"],
    scalers=scalers,
    phase="train",
    s3_use_simplecache=True,
    s3_cache_dir="/tmp/helio_s3_cache",
    #### Put your downstream (DS) specific parameters below this line
    return_surya_stack=True,
    max_number_of_samples=64,        # keep small while debugging; increase later
    ds_spectra_csv_path=ds_spectra_csv_path,
    ds_time_column="timestamp",
    ds_time_tolerance="100d",
    ds_match_direction="forward",
)

# Validation dataset (EVE) - follow template pattern
val_dataset = EVESpectraDSDataset(
    index_path=config["data"]["valid_data_path"],
    time_delta_input_minutes=config["data"]["time_delta_input_minutes"],
    time_delta_target_minutes=config["data"]["time_delta_target_minutes"],
    n_input_timestamps=config["model"]["time_embedding"]["time_dim"],
    rollout_steps=config["rollout_steps"],
    channels=config["data"]["channels"],
    drop_hmi_probability=config["drop_hmi_probability"],
    use_latitude_in_learned_flow=config["use_latitude_in_learned_flow"],
    scalers=scalers,
    phase="val",
    s3_use_simplecache=True,
    s3_cache_dir="/tmp/helio_s3_cache",
    return_surya_stack=True,
    max_number_of_samples=64,
    ds_spectra_csv_path=ds_spectra_csv_path,
    ds_time_column="timestamp",
    ds_time_tolerance="100d",
    ds_match_direction="forward",
)

len(train_dataset), len(val_dataset)


(64, 3)

We also intialize separate training and validation dataloaders.   Since we are working in a shared environment.  Using multiprocessing_context="spawn" helps avoid lockups.

In [13]:
batch_size = 2

train_data_loader = DataLoader(
                dataset=train_dataset,
                batch_size=batch_size,
                shuffle=True,
                num_workers=4,
                multiprocessing_context="spawn",
                persistent_workers=True,
                pin_memory=True,
            )

val_data_loader = DataLoader(
                dataset=val_dataset,
                batch_size=batch_size,
                num_workers=4,
                multiprocessing_context="spawn",
                persistent_workers=True,
                pin_memory=True,
            )

batch = next(iter(train_data_loader))
batch.keys()

dict_keys(['ts', 'time_delta_input', 'forecast', 'lead_time_delta', 'ds_index'])

## Define simple baseline model

Defining a simple baseline is important to understand what value is bringing the AI model to the problem.  

It is always very good to have a very simple baseline model.  Ideally one that cannot overfit the data.  This is a very good way of really measuring the value added of complex models.   Classical machine learning excels here:

- Regressions and logistic regressions.
- Climatological averages.
- Persistance.
- Simple transformations.

Simple models avoid excesively optimistic assessments of the capatiblities of a complex models and for many problems are actually remarkably hard to beat.

In this example we define a simple regression acting on the intensity of each channel.  Note that we invert the normalization to deal with strictly positive quantities.  As with the dataset we will be importing the model from a module so that we can use it within training scripts later on.

In [16]:
from downstream_apps.template.models.simple_baseline import RegressionFlareModel
from downstream_apps.template.models.eve_baseline import RegressionEVEModel
import numpy as np


We can now test that this model manipulates a batch as expected and returns an estimate of flare intensity

Note that the simple regression model definition requires knowing the number of channels and timesteps so here we pull that information from the configuration intializing the model.

In [17]:
# Infer output dimension (number of wavelength bins) from the dataset label
sample = train_dataset[0]
output_dim = int(np.asarray(sample["forecast"]).shape[0])

# Input dimension should match the template convention: C * T (after spatial pooling)
channel_order = config["data"]["channels"]
n_channels = len(channel_order)
n_input_timestamps = config["model"]["time_embedding"]["time_dim"]
input_dim = n_channels * n_input_timestamps

print("n_channels:", n_channels)
print("n_input_timestamps:", n_input_timestamps)
print("input_dim:", input_dim)
print("output_dim (spectrum length):", output_dim)

eve_model = RegressionEVEModel(
    input_dim=input_dim,
    output_dim=output_dim,
    channel_order=channel_order,
    scalers=scalers,
)

eve_model


n_channels: 13
n_input_timestamps: 1
input_dim: 13
output_dim (spectrum length): 1343


RegressionEVEModel(
  (linear): Linear(in_features=13, out_features=1343, bias=True)
)

Now we can pass the input stack 'ts' to the model to transform it into our regression output.   Note that since this model has not been trained and was initalized randomly.  The output here has no real meaning.  It only acts as a test that our model forward doesn't have dimension problems.

Dimension problemns are the dominant source of error in this kind of work.

Note that our output has now the size of our batch.

In [26]:
batch = next(iter(train_data_loader))

eve_model.eval()
with torch.no_grad():
    output = eve_model(batch)

print(output.shape, batch["forecast"].shape)
print("target shape:", tuple(batch["forecast"].shape))

# Expect: yhat = (B, output_dim), target = (B, output_dim)


torch.Size([2, 1343]) torch.Size([2, 1343])
target shape: (2, 1343)


## Define your metrics

Metrics are a very important part of training AI models.   They provide your models with the quantitification of error, which in turn shifts the weights towards better pefrorming models.  They also provide a way for you to monitor performance, identify overfitting, and quantify value added. 

We now initialize the metrics class which allows you to control what metrics do you want to use as "loss" (i.e. the metrics that backpropagate through your model) and which ones for monitoring performance.  As with other components, this takes the form of a loaded module that can be later use in a training script

In [46]:
from downstream_apps.template.metrics.template_metrics import FlareMetrics
from downstream_apps.template.metrics.eve_metrics import EVEMetrics

In [47]:
import inspect
print(inspect.signature(EVEMetrics.__init__))
print(EVEMetrics.__module__)

(self, mode: str)
downstream_apps.template.metrics.eve_metrics


In [44]:
# Use the EVE metrics class you created (EVEMetrics) instead of FlareMetrics
train_loss_metrics = EVEMetrics("train_loss", silent=False)
train_evaluation_metrics = EVEMetrics("train_metrics", silent=False)
validation_evaluation_metrics = EVEMetrics("val_metrics", silent=False)


TypeError: EVEMetrics.__init__() got an unexpected keyword argument 'silent'

Now they can be evaluated in our model's output and our ground truth.   First the loss that actually will backpropagate, in this case Mean Squared Errror

In [37]:
train_loss_metrics(output, batch["forecast"])

ModuleNotFoundError: No module named 'torch._subclasses.schema_check_mode'

Then a training evaluation that will not backpropagate and inform our model, but that we can keep an eye on. Note that reporting lots of metrics during training will slow the training process.  I'm including it her as an example, but oftentimes is better to put the diagnostics only in the validation evaluation metrics.

Here we are caclulating the Root Relative Squared Error https://lightning.ai/docs/torchmetrics/stable/regression/rse.html 

A value below one means the prediction is better than predicting the average.  It is unlikely that this metric will be lower than one with a randomly initialized model

In [15]:
train_evaluation_metrics(output, batch["forecast"])

({'rrse': tensor(2.6078)}, [1])

In the validation evaluation metrics we report both MSE and RRSE

In [16]:
validation_evaluation_metrics(output, batch["forecast"])

({'mse': tensor(0.6465), 'rrse': tensor(2.6078)}, [1, 1])

## Define your PyTorch ligthning module

In this workshop we will use PyTorch lightning to train our models.  PyTorch lighting reduces the amount of code required to implement a training loop in comparison to PyTorch (at the expense of control and versatility).  

Opening the FlareLightningModule shows a simple Lightning model implementation.  It consists of:

- An initialization of the class (metrics, model, and learning rate).
- The forward code that runs evaluation of the model.
- Training and validation steps.
- Configuration of optimizers.

In [17]:
from downstream_apps.template.lightning_modules.pl_simple_baseline import FlareLightningModule

## Set your global seeds

Since training AI models generally uses stochastic gradient descent, it is a good idea to fix your random seeds so that your training exercise is reproducible.    

In [18]:
L.seed_everything(42, workers=True)

Seed set to 42


42

## Intialize Lightning module

Now we properly initalize the Lightning module to enable training, including passing the dictionary of metrics

In [19]:
metrics = {'train_loss': train_loss_metrics,
           'train_metrics': train_evaluation_metrics,
           'val_metrics': validation_evaluation_metrics}

learning_rate = 1e-3
lit_model = FlareLightningModule(model, metrics, lr=learning_rate, batch_size=batch_size)

## Logging

In order to properly compare experiments against each other, it is very useful to log evaluation metrics in a place where they can be compared against other training runs.  In this workshop we will use Weights and Biases (WandB). 

The first time you run WandB in a machine it will ask you to login to WandB.  You should have received an invitation to our project.  In order to login you must:

- Select option 2 (existing account).   In VScode the dialog opens a box at the top of your screen.
- Click on get API Key (this will open a browser).
- Generate API Key.
- Paste it in the dialog box at the top of your VSCode

In [20]:
project_name = "template_flare_regression"
run_name = "baseline_experiment_1"

wandb_logger = WandbLogger(
    entity="surya_handson",
    project=project_name,
    name=run_name,
    log_model=False,
    save_dir="./wandb/wandb_tmp",
)

csv_logger = CSVLogger("runs", name="simple_flare")

## Initialize trainer

With the loggers done, now the trainer needs to be defined.  The trainer defines several properties of your training run. Here we define:

- The max number of epochs (one epoch represents your model seeing your entire training dataset).
- Define where the training run will take place (auto uses the GPU if possible, if not, CPU).
- The loggers.
- The callbacks (here we save the model with the lowest validation loss).
- Logging frequency (because we are working with a small dataset it needs to be small).

In [21]:
max_epochs = 2

# -------------------------------------------------------------------------
# Trainer
# -------------------------------------------------------------------------
trainer = L.Trainer(
    max_epochs=max_epochs,
    accelerator="auto",
    devices="auto",
    logger=[wandb_logger, csv_logger],
    callbacks=[
        ModelCheckpoint(
            monitor="val_loss",
            mode="min",
            save_top_k=1,
        )
    ],
    log_every_n_steps=2,
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores


## Fit the model

Finally we fit the model.  We pass the Lighting module, and our dataloaders.

In [22]:
trainer.fit(lit_model, train_data_loader, val_data_loader)

wandb: WARNING The anonymous setting has no effect and will be removed in a future version.
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice:wandb: You chose 'Use an existing W&B account'
wandb: Logging into https://api.wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: Find your API key here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:wandb: Paste an API key from your profile and hit enter:wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /home/shahbahauddin/.netrc
wandb: Currently logged in as: shah-bahauddin (shah-bahauddin-university-of-colorado-boulder) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [2]

  | Name  | Type                 | Params | Mode  | FLOPs
---------------------------------------------------------------
0 | model | RegressionFlareModel | 14     | train | 0    
---------------------------------------------------------------
14        Trainable params
0         Non-trainable params
14        Total params
0.000     Total estimated model params size (MB)
2         Modules in train mode
0         Modules in eval mode
0         Total Flops


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=2` reached.


## Conclusion

With this we have now integrated our dataset, dataloaders, metrics, and baseline into an end-2-end training loop.  The next step is to substitute the simple model with Surya.